In [0]:
df_cust=spark.read.csv('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/Customer/customer.csv',header=False,inferSchema=True,sep=',').toDF('id','name','age','city','plan')

df_cust.display()
df_cust.show(10,False)
df_cust.show(10,True)
df_cust.take(5)
display(df_cust.sample(.1))




In [0]:
df_cust.printSchema()
print(df_cust.columns)
print(df_cust.dtypes)

In [0]:
print(df_cust.count())
print(df_cust.distinct().count())
print(df_cust.dropDuplicates().count())
display(df_cust.describe())
display(df_cust.summary())

In [0]:
str1="ide string,name string,age string,city string,plan string"
str2="ids integer,name string,age string,city string,plan string"

df_cust2=spark.read.schema(str2).csv('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/Customer/customer.csv',header=False,sep=',')

df_cust3=spark.read.schema(str1).csv('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/Customer/customer.csv',header=False,sep=',')

df_cust4=df_cust2.union(df_cust3)

df_cust4.display()
df_cust4.printSchema()

In [0]:
from pyspark.sql.types import StructType,StructField,IntegerType,StringType

sch= StructType([StructField('id',IntegerType(),True),StructField('name',StringType(),True),StructField('age',IntegerType(),True),StructField('city',StringType(),True),StructField('plan',StringType(),True)])

df1=spark.read.schema(sch).csv('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/Customer/customer.csv',header=False,sep=',',mode='permissive')

df2=spark.read.schema(sch).csv('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/Customer/customer.csv',header=False,sep=',',mode='DROPMALFORMED')

df1.display()
df2.display()
df_cust3.display()


In [0]:
from pyspark.sql.types import StructType,StructField,IntegerType,StringType

sch3= StructType([StructField('Id',IntegerType()),StructField('Name',StringType()),StructField('Age',IntegerType()),StructField('City',StringType()),StructField('Plan',StringType()),StructField('CorruptedRecords',StringType())])

df6=spark.read.schema(sch3).csv('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/Customer/customer.csv',header=False,sep=',',columnNameOfCorruptRecord='CorruptedRecords')

display(df6)


